In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.append('/app')

In [3]:
from create_dummy_data import create_dummy_data

In [5]:
await create_dummy_data()

(UUID('4039d4f0-a8ab-4aa0-b412-f6caa405b251'),
 'f2293266-1c6d-4887-a7c0-de1352e4d8cc')

In [3]:
from core import generic_call,assistant_msg,user_msg,system_msg

In [4]:
from prompts import Prompts

In [5]:
prompts = Prompts()

In [6]:
prompts.create_dummy_trasncript

"You are to create a detailed and realistic transcript of a meeting that lasts between 40 to 60 minutes. The transcript should be based on the meeting summary provided below. Your goal is to produce a genuine and engaging conversation that reflects the distinct characters, their personalities, roles, and the topics discussed during the meeting.\n\nGuidelines:\n\nParticipants:\n\nInclude all attendees mentioned in the summary.\nGive each participant a distinct voice and personality consistent with their role.\nReflect their communication styles (e.g., formal, informal, technical, enthusiastic).\nHave participants reference other colleagues, departments, or stakeholders relevant to the discussion.\nStructure:\n\nBegin with a brief introduction, including greetings and any small talk.\nIncorporate small stories or anecdotes relevant to the meeting topics.\nProgress into the main agenda items as outlined in the summary.\nInclude discussions, debates, questions, and clarifications.\nConclud

In [7]:
from pydantic_models import DummyTranscript

In [15]:
from datetime import datetime, timedelta
import asyncio
import uuid
from sqlalchemy import select
from psql_models import User, Meeting, UserMeeting, Speaker, meeting_speaker_association, AccessLevel
from psql_helpers import get_session
from pydantic_models import DummyTranscript
from core import system_msg, user_msg
from prompts import Prompts

async def create_dummy_user():
    async with get_session() as session:
        # Create dummy user if not exists
        dummy_user_id = uuid.uuid4()
        user_query = select(User).where(User.email == "maya_petel@example.com")
        existing_user = await session.execute(user_query)
        user = existing_user.scalar_one_or_none()
        
        if not user:
            user = User(
                id=dummy_user_id,
                email="maya_petel@example.com",
                username="Maya Patel",
                first_name="Maya",
                last_name="Patel"
            )
            session.add(user)
            await session.commit()
        
        return user
    
user = await create_dummy_user()

In [17]:
user.id

UUID('d4a321c8-6e10-449e-85f2-42124a95cfad')

In [18]:
inputs =prompts.dummy_input_prompts.split('|||')

In [20]:
inputs = [[system_msg(prompts.create_dummy_trasncript),
       user_msg(prompts.dummy_user),
       user_msg(i)] for i in inputs]

In [21]:
inputs[0]

[Msg(role='system', content="You are to create a detailed and realistic transcript of a meeting that lasts between 40 to 60 minutes. The transcript should be based on the meeting summary provided below. Your goal is to produce a genuine and engaging conversation that reflects the distinct characters, their personalities, roles, and the topics discussed during the meeting.\n\nGuidelines:\n\nParticipants:\n\nInclude all attendees mentioned in the summary.\nGive each participant a distinct voice and personality consistent with their role.\nReflect their communication styles (e.g., formal, informal, technical, enthusiastic).\nHave participants reference other colleagues, departments, or stakeholders relevant to the discussion.\nStructure:\n\nBegin with a brief introduction, including greetings and any small talk.\nIncorporate small stories or anecdotes relevant to the meeting topics.\nProgress into the main agenda items as outlined in the summary.\nInclude discussions, debates, questions, 

In [22]:
r = await DummyTranscript.call(inputs[0])
r


DummyTranscript(entries=[DummyTranscriptEntry(speaker='Maya Patel', timestamp=datetime.datetime(2024, 3, 1, 8, 0), content='Good morning, everyone! I hope you all are doing well. It’s a bit rainy here in Seattle, but that’s typical for this time of year. How’s the weather in Germany?'), DummyTranscriptEntry(speaker='Lukas Schmidt', timestamp=datetime.datetime(2024, 3, 1, 8, 0, 15), content='Good morning, Maya! It’s actually quite sunny here today, which is a nice change. We’re enjoying it while it lasts!'), DummyTranscriptEntry(speaker='Anja Müller', timestamp=datetime.datetime(2024, 3, 1, 8, 0, 30), content='Morning! I wish we could send some of this sunshine your way. It’s been a bit dreary lately.'), DummyTranscriptEntry(speaker='Stefan Weber', timestamp=datetime.datetime(2024, 3, 1, 8, 0, 45), content='Good morning, everyone! I’m just glad we’re not dealing with snow. Let’s keep it that way!'), DummyTranscriptEntry(speaker='Maya Patel', timestamp=datetime.datetime(2024, 3, 1, 8, 1)

In [23]:
r.entries

[DummyTranscriptEntry(speaker='Maya Patel', timestamp=datetime.datetime(2024, 3, 1, 8, 0), content='Good morning, everyone! I hope you all are doing well. It’s a bit rainy here in Seattle, but that’s typical for this time of year. How’s the weather in Germany?'),
 DummyTranscriptEntry(speaker='Lukas Schmidt', timestamp=datetime.datetime(2024, 3, 1, 8, 0, 15), content='Good morning, Maya! It’s actually quite sunny here today, which is a nice change. We’re enjoying it while it lasts!'),
 DummyTranscriptEntry(speaker='Anja Müller', timestamp=datetime.datetime(2024, 3, 1, 8, 0, 30), content='Morning! I wish we could send some of this sunshine your way. It’s been a bit dreary lately.'),
 DummyTranscriptEntry(speaker='Stefan Weber', timestamp=datetime.datetime(2024, 3, 1, 8, 0, 45), content='Good morning, everyone! I’m just glad we’re not dealing with snow. Let’s keep it that way!'),
 DummyTranscriptEntry(speaker='Maya Patel', timestamp=datetime.datetime(2024, 3, 1, 8, 1), content='Absolutel

In [ ]:
meeting = Meeting(
                meeting_id=uuid.uuid4(),
                transcript=str([{
                    'speaker': entry.speaker,
                    'timestamp': entry.timestamp.isoformat(),
                    'content': entry.content
                } for entry in transcript.entries),
                timestamp=base_date + timedelta(days=i),
                meeting_name=f"Dummy Meeting {i+1}",
                meeting_summary="Sample meeting about product feedback",
                is_indexed=True
            )
session.add(meeting)
await session.flush()